In [5]:
!pip install yfinance --upgrade --no-cache-dir
!pip install requests --upgrade
!pip install urllib3 --upgrade

In [6]:
import yfinance as yf
import numpy
import pandas
import requests 
import json

from google.colab import files
from google.colab import drive
import os

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
'''
Task Outline

a. Obtain CSV of all stock tickers ✅
b. Conduct Analysis for Bollinger Bands and Keltner Channels in mechanism for detecting consolidation ✅
c. Analysis for the Zeta Model
d. conglomerate them all into one stock screener than prints out the list of all the stock tickers that we should run Black Scholes and Monte Carlo on

'''

In [ ]:
# PART A

data = pandas.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
data[0].head()
data = data[0]['Symbol']

data.to_csv('S&P500tickers.csv', mode = 'w', index = False)
files.download('S&P500tickers.csv')

In [ ]:
with open('S&P500tickers.csv') as f:
  symbols = f.read().splitlines()
  symbols.pop(0)


  for i in symbols:
      ticker_dta = yfinance.download(i, start = "2022-04-02", end = "2023-02-11") # arbitrarily chosen 45 weeks (6 -> 65 weeks)
      drive.mount('/drive')
      ticker_dta.to_csv('/drive/My Drive/stock_screener/{}.csv'.format(i), encoding='utf-8', index=False)
  

In [ ]:
from IPython.lib.security import passwd
drive.mount('/drive')
dir = '/drive/MyDrive/stock_screener/'

def s_mov_avg(df, lookback_window):
  sma = df['Close'].rolling(window = lookback_window, min_periods = 1).mean()  
  std = df['Close'].rolling(window = lookback_window, min_periods = 1).std()
  return sma, std

def atr(df, lookback_window):
  atr_ = df['True Range'].rolling(window = lookback_window, min_periods = 1).mean()
  return atr_


def in_consolidation(curr_df):
  return (curr_df['Lower Bollinger Band'] > curr_df['Lower Keltner Channel']) \
          and (curr_df['Upper Bollinger Band']  < curr_df['Upper Keltner Channel'])

symbols_of_companies = []
for file in os.listdir(dir):
  symbol = file.split()[0]
  symbols_of_companies.append(symbol[:-4])
  curr_df = pandas.read_csv(dir + "{}".format(file))
  if not curr_df.empty:
    sma, std = s_mov_avg(curr_df, 20)
    curr_df['Upper Bollinger Band'] = sma + 2 * std
    curr_df['Lower Bollinger Band'] = sma - 2 * std
    curr_df['Middle Bollinger Band'] = sma
    curr_df['Standard Deviation'] = std

    curr_df["True Range"] = abs(curr_df["High"] - curr_df["Low"])
    curr_df['Average True Range'] = atr(curr_df, 20)

    curr_df['Upper Keltner Channel'] = curr_df['Middle Bollinger Band'] + 1.2 * curr_df['Average True Range']
    curr_df['Lower Keltner Channel'] = curr_df['Middle Bollinger Band'] - 1.2 * curr_df['Average True Range'] 

    # in consolidation

    curr_df['Consolidation Status'] = curr_df.apply(in_consolidation, axis = 1)


In [ ]:
symbols_of_companies
consolidated_status = curr_df['Consolidation Status'].to_list()

mapped_to_consol = dict(zip(symbols_of_companies, consolidated_status))

tb_deleted = []
for k,v in mapped_to_consol.items():
  if not v:
    tb_deleted.append(k)

for i in tb_deleted:
  del mapped_to_consol[i]



In [ ]:
tckr_lst = list(mapped_to_consol.keys())


FMP_API_KEY = "1b000ffcda4a16a7cf6ea29445140466"


def get_symbol_mkt_cap(symbol):
    
    session = requests.Session()

    request = f"https://financialmodelingprep.com/api/v3/market-capitalization/{symbol}?apikey={FMP_API_KEY}".replace(" ", "")

    r = session.get(request)

    if r.status_code == requests.codes.ok:
      
        df = pandas.DataFrame(json.loads(r.text))

    return df
        
    
def get_balance_sheet(symbol, lookback):
    
    session = requests.Session()
    
    request = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?limit={lookback}&apikey={FMP_API_KEY}".replace(" ", "")
    
    r = session.get(request)
    
    if r.status_code == requests.codes.ok:
        
        df = pandas.DataFrame(json.loads(r.text))
        
    return df


def get_income_statement(symbol, lookback):
    
    session = requests.Session()
    
    request = f"https://financialmodelingprep.com/api/v3/income-statement/{symbol}?limit={lookback}&apikey={FMP_API_KEY}".replace(" ", "")
    
    r = session.get(request)
    
    if r.status_code == requests.codes.ok:
        
        df = pandas.DataFrame(json.loads(r.text))
        
    return df

def get_key_metrics(symbol, lookback):
    
    session = requests.Session()
    
    request = f"https://financialmodelingprep.com/api/v3/key-metrics-ttm/{symbol}?apikey={FMP_API_KEY}&limit={lookback}".replace(" ", "")
    
    r = session.get(request)
    
    if r.status_code == requests.codes.ok:
        
        df = pandas.DataFrame(json.loads(r.text))
        
    return df


NameError: name 'mapped_to_consol' is not defined

In [ ]:
# extracts company data for zeta model calculation

zeta_lst = []

for i in tckr_lst:
  market_cap = get_symbol_mkt_cap(symbol = i)
  balance_sheet = get_balance_sheet(symbol= i, lookback=1)
  income_statement = get_income_statement(symbol = i, lookback = 1)
  key_metrics = get_key_metrics(symbol=i, lookback=1)


  coefficient_A = key_metrics["workingCapitalTTM"] / balance_sheet["totalAssets"]
  coefficient_B = balance_sheet["retainedEarnings"] / balance_sheet["totalAssets"]
  coefficient_C = (income_statement["ebitda"] - income_statement['depreciationAndAmortization']) / balance_sheet["totalAssets"]
  coefficient_D = market_cap['marketCap'] / balance_sheet["totalLiabilities"]
  coefficient_E =  income_statement["revenue"] / balance_sheet["totalAssets"]

  # fidget with the coefficients
  zeta_score = 1.2*coefficient_A + 1.4*coefficient_B + 3.3*coefficient_C + 0.6*coefficient_D + 1.0*coefficient_E
  # print(zeta_score)
  zeta_lst.append((i, zeta_score))



NameError: name 'tckr_lst' is not defined

In [ ]:
ans = []
for i in zeta_lst:
  ans.append((i[0], i[1].values))


ans = [(x, float(y)) for x,y in ans]

# Z score of >= 3 indicates prime credibility (top 1%) thus based on this ticker ADBE and ADM are indicators of possible stocks


stocks_to_invest = []
counter = len(ans)

while counter >= 1:
  if ans[counter - 1][1] >= 3:
    stocks_to_invest.append(ans[counter - 1][0])
  
  counter -=1

# ***********************
print(stocks_to_invest)
# ***********************

[]
